In [78]:
import pickle
import numpy as np

import os
import sys
from scipy import optimize
from torch.utils.data import DataLoader, Dataset
from data_loaders import *
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import ExtraTreesRegressor
from tqdm import tqdm

In [79]:
def load_index(observed_values,seed,gt_masks):

    start_sim = True
    while start_sim:

        N, D = observed_values.shape

        dl = D - 1
        
        indlist = np.arange(N)

        np.random.seed(seed + 1)
        np.random.shuffle(indlist)

        tmp_ratio = 1 / 5
        start = (int)((5 - 1) * N * tmp_ratio)
        
        end = (int)(5 * N * tmp_ratio)

        test_index = indlist[start:end]
        remain_index = np.delete(indlist, np.arange(start, end))

        np.random.shuffle(remain_index)


        # Modify here to change train,valid ratio
        num_train = (int)(len(remain_index) * 0.9)
        train_index = remain_index[:num_train]
        valid_index = remain_index[num_train:]



        print(len(train_index),len(test_index))
        
        Xtrain = observed_values[train_index]
        Xtest = observed_values[test_index]

        

        Xtrain_mask = gt_masks[train_index]
        Xtest_mask = gt_masks[test_index]

        zero_percentage_train = (Xtrain_mask == 0).mean() * 100
        zero_percentage_test = (Xtest_mask == 0).mean() * 100

        start_sim = False
        # if zero_percentage_test > 0:
        #     start_sim = False
        # else:
        #     seed += 1

    return Xtrain,Xtest,Xtrain_mask,Xtest_mask

In [80]:
datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

datalist = [
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = ["logistic","diffuse","quantile"]


In [81]:
# dataname = "banknote"#,,
# dataname = "concrete_compression"
# #dataname = "wine_quality_white"
datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = ["quantile"]

seed = 1
nfold = 5
# #missingtype = "logistic"
# missingtype = "diffuse"
# #missingtype = "self_mask"

# missing_rule = load_json_file("diffuse_ratio.json")
# #missing_rule = load_json_file("q_ratio.json")

for dataname in tqdm(datalist):
    
    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")
            #missing_rule = load_json_file("complete.json")
        

        rule_list = []
        mean_list = []
        mice_list = []
        knn_list = []
        missforest_list = []


        for rule_name in missing_rule:
            rule = missing_rule[rule_name]
            print("Current Rule",rule_name, dataname)
            # Create folder
            # Every loader contains "observed_data", "observed_mask", "gt_mask", "timepoints"
            np.random.seed(seed)

            dataset_path = f"datasets/{dataname}/data.csv"
            processed_data_path = (
                f"datasets/{dataname}/{missingtype}-{rule_name}_seed-{seed}.pk"
            )
            processed_data_path_norm = (
                f"datasets/{dataname}/{missingtype}-{rule_name}_seed-{seed}_max-min_norm.pk"
            )


            if os.path.isfile(processed_data_path_norm):
                with open(processed_data_path_norm, "rb") as f:
                    observed_values, observed_masks, gt_masks, eval_length = pickle.load(
                        f
                    )
            else:
                print("no data")
                break
            
            # Calculate the percentage of zeros
            # zero_percentage = (gt_masks == 0).mean() * 100

            # print(f"0 的占比: {zero_percentage}%")

            Xtrain,Xtest,Xtrain_mask,Xtest_mask = load_index(observed_values,seed,gt_masks)
            
        #     # print("train_index",len(train_idx))
        #     # print("test_index",len(test_idx))


            # Xtrain = observed_values[train_idx]
            # Xtest = observed_values[test_idx]

            

            # Xtrain_mask = gt_masks[train_idx]
            # Xtest_mask = gt_masks[test_idx]

            # zero_percentage_train = (Xtrain_mask == 0).mean() * 100
            # zero_percentage_test = (Xtest_mask == 0).mean() * 100

            Xnan = Xtrain.copy()
            Xz = Xtrain.copy()
            Xnan[Xtrain_mask == 0] = np.nan
            Xz[Xtrain_mask == 0] = 0
            
            
            X_test_nan = Xtest.copy()
            X_test_z = Xtest.copy()
            X_test_nan[Xtest_mask == 0] = np.nan
            X_test_z[Xtest_mask == 0] = 0

            nan_columns = np.all(np.isnan(Xnan), axis=0)
            if np.any(nan_columns):
                random_value = np.random.uniform(0, 1)
                Xnan[:, nan_columns] = random_value 


            nan_columns = np.all(np.isnan(X_test_nan), axis=0)
            if np.any(nan_columns):
                random_value = np.random.uniform(0, 1)
                X_test_nan[:, nan_columns] = random_value 


            mean, Ximp = mean_imputer(Xnan,X_test_nan,Xtest,Xtest_mask)
            pd.DataFrame(Ximp).to_csv("results/baselines/Imputation_{}_{}_{}.csv".format(dataname,missingtype,rule_name),index=False)
            mice, Ximp = mice_imputer(Xnan,X_test_nan,Xtest,Xtest_mask)
            pd.DataFrame(Ximp).to_csv("results/baselines/Imputation_{}_{}_{}.csv".format(dataname,missingtype,rule_name),index=False)
            knn, Ximp = knn_imputer(Xnan,X_test_nan,Xtest,Xtest_mask)
            pd.DataFrame(Ximp).to_csv("results/baselines/Imputation_{}_{}_{}.csv".format(dataname,missingtype,rule_name),index=False)
            missforest, Ximp = missforest_imputer(Xnan,X_test_nan,Xtest,Xtest_mask)
            pd.DataFrame(Ximp).to_csv("results/baselines/Imputation_{}_{}_{}.csv".format(dataname,missingtype,rule_name),index=False)
            
            rule_list.append(rule_name)
            mean_list.append(mean)
            mice_list.append(mice)
            knn_list.append(knn)
            missforest_list.append(missforest)


        result = pd.DataFrame({"Missing_Rule":rule_list,"Mean":mean_list,"Mice":mice_list,"Missforest":missforest_list,"KNN":knn_list})
        result.to_csv("results/baselines/RMSE_{}_{}.csv".format(dataname,missingtype),index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Current Rule Q1_Q2_0.25 banknote
986 275


e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Current Rule Q1_Q3_0.25 banknote
986 275


e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Current Rule Q3_1.0 yacht_hydrodynamics
221 62


e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Current Rule Q4_1.0 yacht_hydrodynamics
221 62


e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
100%|██████████| 10/10 [1:21:36<00:00, 489.69s/it]


In [67]:
def mean_imputer(train_nan,test_nan,test_real,test_mask):

    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(train_nan)
    Ximp = imp.transform(test_nan)
    column_rmse_list = column_rmse(Ximp, test_real, test_mask)
    return np.sqrt(np.sum((test_real - Ximp) ** 2 * (1 - test_mask)) / np.sum(1 - test_mask)), Ximp

def mice_imputer(train_nan,test_nan,test_real,test_mask):

    imp = IterativeImputer(random_state=0, sample_posterior=True)
    imp.fit(train_nan)
    Ximp = imp.transform(test_nan)
    return np.sqrt(np.sum((test_real - Ximp) ** 2 * (1 - test_mask)) / np.sum(1 - test_mask)), Ximp


def knn_imputer(train_nan,test_nan,test_real,test_mask):

    imp = KNNImputer(n_neighbors=2)
    imp.fit(train_nan)
    Ximp = imp.transform(test_nan)
    return np.sqrt(np.sum((test_real - Ximp) ** 2 * (1 - test_mask)) / np.sum(1 - test_mask)), Ximp


def missforest_imputer(train_nan,test_nan,test_real,test_mask):
    impute_estimator = ExtraTreesRegressor(n_estimators=10, random_state=0)

    imp = IterativeImputer(random_state=0, estimator=impute_estimator)
    imp.fit(train_nan)
    Ximp = imp.transform(test_nan)
    return np.sqrt(np.sum((test_real - Ximp) ** 2 * (1 - test_mask)) / np.sum(1 - test_mask)), Ximp


def column_rmse(imputed_data, real_data, mask):
    column_rmse_list = []
    for col_idx in range(imputed_data.shape[1]):
        imputed_col = imputed_data[:, col_idx]
        real_col = real_data[:, col_idx]
        mask_col = mask[:, col_idx]
        
        # Only consider values where mask is 0 (i.e., missing values)
        imputed_col = imputed_col[mask_col == 0]
        real_col = real_col[mask_col == 0]
        
        rmse = np.sqrt(np.mean((real_col - imputed_col) ** 2))
        column_rmse_list.append(rmse)
    return column_rmse_list


In [77]:
datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = ["quantile"]


for dataname in tqdm(datalist):
    
    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")
            #missing_rule = load_json_file("complete.json")
        

        rule_list = []
        mean_list = []
        mice_list = []
        knn_list = []
        missforest_list = []


        for rule_name in missing_rule:
            rule = missing_rule[rule_name]
            # Create folder
            # Every loader contains "observed_data", "observed_mask", "gt_mask", "timepoints"
            np.random.seed(seed)

            dataset_path = f"datasets/{dataname}/data.csv"
            processed_data_path = (
                f"datasets/{dataname}/{missingtype}-{rule_name}_seed-{seed}.pk"
            )
            processed_data_path_norm = (
                f"datasets/{dataname}/{missingtype}-{rule_name}_seed-{seed}_max-min_norm.pk"
            )


            if os.path.isfile(processed_data_path_norm):
                with open(processed_data_path_norm, "rb") as f:
                    observed_values, observed_masks, gt_masks, eval_length = pickle.load(
                        f
                    )
            else:
                print("no data")
                break
            
            #Calculate the percentage of zeros
            zero_percentage = (gt_masks == 0).mean() * 100

            print(f"{dataname} {rule_name} 0 的占比: {zero_percentage}%")


    break



  0%|          | 0/10 [00:00<?, ?it/s]

banknote Q1_Q2_complete 0 的占比: 50.054704595185996%
banknote Q1_Q3_complete 0 的占比: 50.07293946024799%
banknote Q1_Q4_complete 0 的占比: 50.07293946024799%
banknote Q2_Q3_complete 0 的占比: 49.981765134938%
banknote Q2_Q4_complete 0 的占比: 50.09117432530999%
banknote Q3_Q4_complete 0 的占比: 50.054704595185996%
banknote Q1_complete 0 的占比: 25.018234865062%
banknote Q2_complete 0 的占比: 25.036469730123994%
banknote Q3_complete 0 的占比: 25.054704595185996%
banknote Q4_complete 0 的占比: 25.054704595185996%
banknote Q1_Q2_0.25 0 的占比: 37.63676148796499%
banknote Q1_Q3_0.25 0 的占比: 37.63676148796499%
banknote Q1_Q4_0.25 0 的占比: 37.65499635302699%
banknote Q2_Q3_0.25 0 的占比: 37.60029175784099%
banknote Q2_Q4_0.25 0 的占比: 37.65499635302699%
banknote Q3_Q4_0.25 0 的占比: 37.61852662290299%
banknote Q1_0.25 0 的占比: 12.545587162654998%
banknote Q2_0.25 0 的占比: 12.545587162654998%
banknote Q3_0.25 0 的占比: 12.545587162654998%
banknote Q4_0.25 0 的占比: 12.563822027716995%
banknote Q1_Q2_partial 0 的占比: 25.091174325309996%
banknote 